In [2]:
import pandas as pd
from pathlib import Path

project_root = Path().resolve().parent

# Audio + v2_ accel (precomputed)
df_audio = pd.read_csv(project_root / "data" / "processed" / "features_audio.csv")

# Your self-engineered accel features
df_accel = pd.read_csv(project_root / "data" / "processed" / "features.csv")

# Sanity check
assert len(df_audio) == len(df_accel), "Row counts do not match!"


AssertionError: Row counts do not match!

In [6]:
df_audio

,userid,label,v1_mfcc1,v1_mfcc2,v1_mfcc3,v1_mfcc4,v1_mfcc5,v1_mfcc6,v1_mfcc7,v1_mfcc8,...,v2_maxX,v2_maxY,v2_maxZ,v2_corXY,v2_corXZ,v2_corYZ,v2_meanMagnitude,v2_sdMagnitude,v2_auc,v2_meanDif
0,user1,watch_tv,120.078886,3.025171,0.390508,-1.777848,-9.216060,-6.567632,-9.918761,-9.145534,...,0.097412,1.088135,0.227051,0.900163,-0.729511,-0.863478,1.014805,0.060424,70.021543,-0.000364
1,user1,watch_tv,130.243139,2.552278,-6.457879,-7.440159,-3.070106,-3.475476,-5.718819,-2.878174,...,0.703613,1.424805,0.818115,-0.210431,0.319599,-0.722541,1.005713,0.162644,79.451303,-0.006234
2,user1,watch_tv,122.259209,1.970267,-8.980575,-10.222030,-3.564994,-2.496906,-5.584900,-6.730588,...,0.312500,0.972656,-0.070312,-0.018862,-0.093852,-0.305722,1.002809,0.006100,77.216317,0.000019
3,user1,watch_tv,122.609566,3.188740,-7.049397,-10.569875,-0.220824,-0.649459,-4.975825,-6.856288,...,0.349854,1.020020,-0.053223,0.492439,0.273064,-0.023051,1.002878,0.021292,75.215866,-0.000023
4,user1,watch_tv,121.393023,1.338798,-8.706183,-8.654997,-4.935310,-3.401656,-5.031361,-7.957924,...,0.322266,0.955078,-0.076904,-0.226296,-0.217439,-0.053895,1.003427,0.002849,71.243346,-0.000083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,user3,brush_teeth,122.021498,-15.977343,-8.310988,-4.317431,7.649979,-2.668571,-12.292989,4.300571,...,1.121338,0.919922,1.658691,0.406223,-0.764335,-0.491075,1.150296,0.231134,93.173996,0.005270
1382,user3,brush_teeth,117.198158,-11.813229,-11.102937,-6.109680,11.951310,-7.626752,-7.109896,1.786718,...,0.815674,1.324951,1.398193,0.108428,-0.859017,-0.370785,1.110172,0.191855,91.034109,-0.007033
1383,user3,brush_teeth,117.586372,-12.161884,-7.365369,-10.990608,7.411106,0.336587,-8.896576,-6.785887,...,0.787354,1.554443,1.358643,-0.131748,-0.315533,-0.636949,1.112783,0.286620,83.458704,-0.001907
1384,user3,brush_teeth,117.465555,-11.785451,-7.936384,-4.155398,10.444611,-7.911971,-4.873325,2.467046,...,0.778320,1.451660,1.006592,0.256575,-0.397598,-0.744013,1.014352,0.159789,76.076411,0.002174


In [3]:
df_accel

,x_mean,x_std,x_min,x_max,x_energy,x_entropy,y_mean,y_std,y_min,y_max,...,z_std,z_min,z_max,z_energy,z_entropy,sma,activity,sensor,timestamp,user
0,-0.160217,0.174259,-0.586670,0.257812,0.056036,2.008796,0.712187,0.139973,0.422363,1.078369,...,0.485509,-0.692627,0.926270,0.436818,1.278327,1.547447,wash_hands,acc,1469751141723,user2
1,-0.253475,0.208238,-0.975586,0.363525,0.107613,1.797242,0.789051,0.246605,0.228760,1.541016,...,0.666967,-1.700195,1.089111,0.444873,2.089784,1.655164,wash_hands,acc,1469751141723,user2
2,-0.251014,0.195137,-0.975586,0.363525,0.101087,1.710536,0.901313,0.212261,0.228760,1.541016,...,0.526335,-1.700195,1.089111,0.310697,1.909086,1.624003,wash_hands,acc,1469751141723,user2
3,-0.299325,0.101847,-0.643799,-0.042969,0.099968,1.868105,0.886455,0.187987,0.418945,1.528565,...,0.417742,-0.674316,0.949463,0.174604,2.157403,1.557465,wash_hands,acc,1469751141723,user2
4,-0.305087,0.148186,-0.685791,0.202881,0.115037,1.791822,0.898101,0.177048,0.418945,1.528565,...,0.396094,-0.903076,0.636963,0.167189,2.188260,1.568262,wash_hands,acc,1469751141723,user2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,0.647102,0.047407,0.557861,0.733887,0.420988,2.190176,0.665323,0.048164,0.578613,0.751465,...,0.031514,-0.463623,-0.310303,0.146176,2.050623,1.693454,eat_chips,acc,1468367368314,user1
2377,0.637523,0.031515,0.557861,0.708740,0.407429,1.996491,0.675276,0.032195,0.614746,0.751465,...,0.024149,-0.463623,-0.310303,0.151593,1.676943,1.701399,eat_chips,acc,1468367368314,user1
2378,0.665249,0.043402,0.524658,0.747314,0.444440,1.983260,0.646763,0.045647,0.543701,0.765625,...,0.023724,-0.443604,-0.330566,0.146309,2.078155,1.693779,eat_chips,acc,1468367368314,user1
2379,0.637779,0.063051,0.510010,0.747314,0.410737,2.217921,0.670002,0.064429,0.543701,0.779785,...,0.033494,-0.443604,-0.302246,0.145014,2.177817,1.687112,eat_chips,acc,1468367368314,user1


In [5]:
print("audio rows:", len(df_audio))
print("accel rows:", len(df_accel))
print("same user IDs:", (df_audio['userid'].values == df_accel['user'].values).sum())
print("same labels:", (df_audio['label'].values == df_accel['activity'].values).sum())


audio rows: 1386
accel rows: 2381


ValueError: operands could not be broadcast together with shapes (1386,) (2381,) 